In [5]:
import SimpleITK as sitk
import numpy as np
import os

# Input and output paths
input_folder = r"C:\Users\R c\Documents\CT_pipeline_Portfolio\raw_data"
output_folder = r"C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed"
os.makedirs(output_folder, exist_ok=True)

# Target spacing (mm)
target_spacing = (1.0, 1.0, 1.0)

def preprocess_ct(image):
    # --- 1. Resample to 1mm spacing ---
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / target_spacing[i])))
        for i in range(3)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetOutputSpacing(target_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    image_resampled = resampler.Execute(image)

    # --- 2. Intensity normalization ---
    array = sitk.GetArrayFromImage(image_resampled).astype(np.float32)
    array = np.clip(array, -1000, 400)  # clamp HU range
    array = (array - array.min()) / (array.max() - array.min())  # scale 0–1
    norm_image = sitk.GetImageFromArray(array)
    norm_image.CopyInformation(image_resampled)

    return norm_image


# Batch preprocess all files
for file in os.listdir(input_folder):
    if file.endswith(".nii.gz"):
        path = os.path.join(input_folder, file)
        img = sitk.ReadImage(path)
        processed = preprocess_ct(img)

        output_path = os.path.join(output_folder, file)
        sitk.WriteImage(processed, output_path)
        print(f"✅ Saved preprocessed file: {output_path}")

print("🎯 All preprocessing completed!")



✅ Saved preprocessed file: C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed\01_patient.nii.gz
✅ Saved preprocessed file: C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed\02_patient.nii.gz
✅ Saved preprocessed file: C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed\03_patient.nii.gz
✅ Saved preprocessed file: C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed\04_patient.nii.gz
✅ Saved preprocessed file: C:\Users\R c\Documents\CT_pipeline_Portfolio\preprocessed\05_patient.nii.gz
🎯 All preprocessing completed!
